In this notebook I will experiment with various preprocessing techniques and test their effect on SVM's performance.



In [143]:
import pandas as pd

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import spacy
spacy.load('en_core_web_sm')

import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc

import numpy as np

import re

import contractions
from textacy.preprocessing import normalize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\olija\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [145]:
sentiment140Df = pd.read_csv("sentiment140.training.1600000.processed.noemoticon.csv", encoding='latin-1', header=None, usecols=[0,5], names=["sentimentScore", "tweet"])
# print("dataframe shape: " + str(sentiment140Df.shape) )

#  Sentiment column (0 = negative, 4 = positive)
print(sentiment140Df.columns.values)
# replace 4 with 1 (0 = negative, 1 = positive)
sentiment140Df['sentimentScore'] = sentiment140Df['sentimentScore'].replace(4,1)
def labelTrainingSentiment(score):
    if score == 0:
        return 'negative'
    if score == 1:
        return 'positive'


sentiment140Df['label'] = sentiment140Df['sentimentScore'].apply(lambda x: labelTrainingSentiment(x))
sentiment140Df.head(5)


['sentimentScore' 'tweet']


,sentimentScore,tweet,label
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",negative
1,0,is upset that he can't update his Facebook by ...,negative
2,0,@Kenichan I dived many times for the ball. Man...,negative
3,0,my whole body feels itchy and like its on fire,negative
4,0,"@nationwideclass no, it's not behaving at all....",negative


In [146]:
# This emoji's dicitonary was taken from:
# https://www.kaggle.com/code/stoicstatic/twitter-sentiment-analysis-for-beginners?scriptVersionId=68181216

emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

htmlSymbols = {"&amp;": "and", "&quot;": '"', "&apos;": "'", "&cent;": "cent", "&pound;": "pound", "&yen;": "yen", "&euro;": "euro",
            "&copy;": "copyright", "&reg;": "registered trademark"}

# Abbreviation dictionary adapted from:
# https://www.socialmediatoday.com/social-networks/sarah-snow/2015-07-08/get-out-your-twittonary-twitter-abbreviations-you-must-know

abbreviationsDict = {'cc': 'carbon-copy', 'cx': 'correction', 'ct': 'cuttweet', 'dm': 'direct message', 'ht': 'heard through', 'mt': 'modified tweet', 
            'prt': 'please retweet', 'rt': 'retweet', 'sp': 'sponsored', 'em': 'email marketing', 'ezine': 'electronic magazine', 'fb': 'facebook', 
            'li': 'linkedin', 'seo': 'search engine optimization', 'sm': 'social media', 'smm': 'social media marketing', 'smo': 'social media optimization', 
            'sn': 'social network', 'sroi': 'social return on investment', 'ugc': 'user generated content', 'ux': 'user experience', 'yt': 'youtube', 
            'abt': 'about', 'afaik': 'as far as i know', 'ayfkmwts': 'are you fucking kidding me with this shit', 'b4': 'before', 'bfn': 'bye for now', 
            'bgd': 'background', 'bh': 'blockhead', 'br': 'best regards', 'btw': 'by the way', 'cd9': 'code 9', 'chk': 'check', 'cul8r': 'see you later', 
            'dam': 'donâ€™t annoy me', 'dd': 'dear daughter', 'df': 'dear fiance', 'ds': 'dear son', 'dyk': 'did you know', 'eml': 'email', 'ema': 'email address', 
            'f2f': 'face to face', 'ftf': 'face to face', 'ff': 'follow friday', 'ffs': "for fuck's sake", 'fml': 'fuck my life', 'fotd': 'find of the day', 
            'ftw': 'for the win', 'fubar': 'fucked up beyond all repair', 'fwiw': "for what it's worth", 'gmafb': 'give me a fucking break', 'gr8': 'great', 
            'gtfooh': 'get the fuck out of here', 'gts': 'guess the song', 'hagn': 'have a good night', 'hand': 'have a nice day', 'hotd': 'headline of the day', 
            'hth': 'hope that helps', 'ic': 'i see', 'icymi': 'in case you missed it', 'idk': "i don't know", 'iirc': 'if i remember correctly', 
            'imho': 'in my humble opinion', 'irl': 'in real life', 'iwsn': 'i want sex now', 'jk': 'joke', 'js': 'just saying', 'jsyk': 'just so you know', 
            'jv': 'joint venture', 'kk': 'okay', 'kyso': 'knock your socks off', 'lbs': 'laughing but serious', 'lhh': 'laugh hella hard', 'lmao': 'laughing my ass off', 
            'lmk': 'let me know', 'lo': 'little one', 'lol': 'laugh out loud', 'mm': 'music monday', 'mirl': 'meet in real life', 'mrjn': 'marijuana', 
            'msm': 'main stream media', 'mtf': 'more to follow', 'nbd': 'no big deal', 'nct': 'nobody cares though', 'nfi': 'no further information or not fucking interested', 
            'nfw': 'no fucking way', 'njoy': 'enjoy', 'nsfw': 'not safe for work', 'nts': 'note to self', 'oh': 'overheard', 'omfg': 'oh my fucking god', 
            'omg': 'oh my god', 'oomf': 'one of my friends', 'orly': 'oh really', 'plmk': 'please let me know', 'pnp': 'party and play', 'poidh': "pictures or it didn't happen", 
            'qotd': 'quote of the day', 're': 'in reply to', 'rlrt': 'real life retweet', 'rtfm': 'read the fucking manual', 'rtq': 'read the question', 'sfw': 'safe for work', 
            'smdh': 'shaking my damn head', 'smh': 'shaking my head', 'snafu': 'situation normal all fucked up', 'srs': 'serious', 'stfu': 'shut the fuck up', 
            'stfw': 'search the fucking web', 'tftf': 'thanks for the follow', 'tftt': 'thanks for this tweet', 'tj': 'tweetjack', 'tl': 'timeline', 'tldr': "too long didn't read", 
            'tmb': 'tweet me back', 'tt': 'trending topic', 'ty': 'thank you', 'tyia': 'thank you in advance', 'tyt': 'take your time', 'tyvm': 'thank you very much', 
            'w': 'with', 'wtv': 'whatever', 'ygtr': 'you got that right', 'ykwim': 'you know what i mean', 'ykyat': "you know you're addicted to", 
            'ymmv': 'your mileage may vary', 'yolo': 'you only live once', 'yoyo': "you're on your own", 'yw': "you're welcome", 'brb' : 'be right back', 'gtg' : 'got to go'}


stops = stopwords.words('english')

def appendStopsNoApostrophes(wordList):
    newList = wordList
    for word in wordList:
        if word.find("'") != -1:
            newList.append(word.replace("'", ""))
            newList.remove(word)
    return newList

stops = appendStopsNoApostrophes(stops)

STOPWORDS = set(stops)

Working on the assumption that URLs, user mentions, and HTML symbols add no meaning, remove or replace these and any extra whitespace from the raw tweet data.

In [147]:
def initialPreprocessTweet(tweet):
     # to lower case
    cleanTweet = tweet.lower()
    # remove user mentions
    cleanTweet = re.sub("(@[A-Za-z0-9_]+)","", cleanTweet)
    # remove urls
    cleanTweet = re.sub("https?://[A-Za-z0-9./]*","", cleanTweet)
    # replace common html symbols
    for key in htmlSymbols.keys():
        cleanTweet = cleanTweet.replace(key, htmlSymbols[key])
    #normalize all whitespace, remove leading and trailing whitespace
    cleanTweet = normalize.whitespace(cleanTweet)    
    return cleanTweet

def replaceEmojis(tweet):
    #replace emojis with text translation
    cleanTweet = tweet
    for key in emojis.keys():
         cleanTweet = cleanTweet.replace(key, emojis[key])
    return cleanTweet

#method to remove stopwords
def removeStops(tweet):
    return " ".join([word for word in tweet.split() if word not in STOPWORDS])

#method to lematize words
nlp = spacy.load("en_core_web_sm",  disable=['parser', 'ner'])
def lemmatize(tweet):
    # lemmatizing words
    doc = nlp(tweet)
    cleanTweet = " ".join([token.lemma_ for token in doc])
    return cleanTweet

#method to remove shortwords
def removeShortWords(tweet):
    return " ".join([word for word in tweet.split() if len(word) > 3])

# expand abbreviations / acronyms - translate common abbreviations to their translation
def translateAbbreviations(tweet):
    cleanTweetList = tweet.split()
    for key in abbreviationsDict.keys():
        cleanTweetList = [abbreviationsDict[key] if item == key else item for item in cleanTweetList]
    return " ".join(cleanTweetList)

# method to expand English language contractions, including negation
def expandContractions(tweet):
    return contractions.fix(tweet)

# method for repeated letter normalization = goooooooood --> good (no english word has more than two consecutive letters the same)
def normalizeRepeatedChars(tweet):
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    cleanTweet = tweet
    for c in alphabet:
        cleanTweet = normalize.repeating_chars(cleanTweet, chars= c, maxn=2)
    return cleanTweet

def removeNonAlphaChars(tweet):
    # remove non alpha characters
    cleanTweet = re.sub("[^a-zA-Z ]","", tweet)
    #normalize all whitespace
    return normalize.whitespace(cleanTweet)    

In [148]:
exampleTweet = "Hello to the WORLD. Today is a gr8 day. i am feeling so goooooood.    There are 12345 reasons to be happy!!!!! :) I can't wait to go surfing.  :(  What are y'all up to today? @world @everyone https://www.google.com :) brb gtg "
cleanedTweet = initialPreprocessTweet(exampleTweet)

print(exampleTweet)
print(cleanedTweet)
print(replaceEmojis(cleanedTweet))
print(removeStops(cleanedTweet))
print(lemmatize(cleanedTweet))
print(removeShortWords(cleanedTweet))
print(translateAbbreviations(cleanedTweet))
print(expandContractions(cleanedTweet))
print(normalizeRepeatedChars(cleanedTweet))
print(removeNonAlphaChars(cleanedTweet))



Hello to the WORLD. Today is a gr8 day. i am feeling so goooooood.    There are 12345 reasons to be happy!!!!! :) I can't wait to go surfing.  :(  What are y'all up to today? @world @everyone https://www.google.com :) brb gtg 
hello to the world. today is a gr8 day. i am feeling so goooooood. there are 12345 reasons to be happy!!!!! :) i can't wait to go surfing. :( what are y'all up to today? :) brb gtg
hello to the world. today is a gr8 day. i am feeling so goooooood. there are 12345 reasons to be happy!!!!! smile i can't wait to go surfing. sad what are y'all up to today? smile brb gtg
hello world. today gr8 day. feeling goooooood. 12345 reasons happy!!!!! :) can't wait go surfing. :( y'all today? :) brb gtg
hello to the world . today be a gr8 day . I be feel so goooooood . there be 12345 reason to be happy ! ! ! ! ! :) I can not wait to go surf . :( what be you all up to today ? :) brb gtg
hello world. today day. feeling goooooood. there 12345 reasons happy!!!!! can't wait surfing.

In [149]:
sentiment140Df['initialPreprocessTweet'] = sentiment140Df['tweet'].apply(lambda x:initialPreprocessTweet(x))
sentiment140Df.head()

,sentimentScore,tweet,label,initialPreprocessTweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",negative,"- awww, that's a bummer. you shoulda got david..."
1,0,is upset that he can't update his Facebook by ...,negative,is upset that he can't update his facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...,negative,i dived many times for the ball. managed to sa...
3,0,my whole body feels itchy and like its on fire,negative,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....",negative,"no, it's not behaving at all. i'm mad. why am ..."


In [151]:
sentiment140Df['initialPreprocessNoPunct'] = sentiment140Df['initialPreprocessTweet'].apply(lambda x:removeNonAlphaChars(x))
sentiment140Df['emojisReplaced'] = sentiment140Df['initialPreprocessTweet'].apply(lambda x:removeNonAlphaChars(replaceEmojis(x)))

In [152]:
sentiment140Df['noStops'] = sentiment140Df['initialPreprocessNoPunct'].apply(lambda x: removeStops(x))
sentiment140Df['lemmatized'] = sentiment140Df['initialPreprocessNoPunct'].apply(lambda x: lemmatize(x))

In [153]:
sentiment140Df['noShortWords'] = sentiment140Df['initialPreprocessNoPunct'].apply(lambda x: removeShortWords(x))
sentiment140Df['translateAbbreviations'] = sentiment140Df['initialPreprocessNoPunct'].apply(lambda x: translateAbbreviations(x))

In [154]:
sentiment140Df['expandContractions'] = sentiment140Df['initialPreprocessNoPunct'].apply(lambda x: expandContractions(x))
sentiment140Df['repeatedCharsNormalized'] = sentiment140Df['initialPreprocessNoPunct'].apply(lambda x: normalizeRepeatedChars(x))
sentiment140Df.head()

,sentimentScore,tweet,label,initialPreprocessTweet,initialPreprocessNoPunct,emojisReplaced,noStops,lemmatized,noShortWords,translateAbbreviations,expandContractions,repeatedCharsNormalized
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",negative,"- awww, that's a bummer. you shoulda got david...",awww thats a bummer you shoulda got david carr...,awww thats a bummer you shoulda got david carr...,awww thats bummer shoulda got david carr third...,awww that s a bummer you shoulda get david car...,awww thats bummer shoulda david carr third,awww thats a bummer you shoulda got david carr...,awww that is a bummer you shoulda got david ca...,aww thats a bummer you shoulda got david carr ...
1,0,is upset that he can't update his Facebook by ...,negative,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,is upset that he cant update his facebook by t...,upset cant update facebook texting might cry r...,be upset that he can not update his facebook b...,upset that cant update facebook texting might ...,is upset that he cant update his facebook by t...,is upset that he cannot update his facebook by...,is upset that he cant update his facebook by t...
2,0,@Kenichan I dived many times for the ball. Man...,negative,i dived many times for the ball. managed to sa...,i dived many times for the ball managed to sav...,i dived many times for the ball managed to sav...,dived many times ball managed save rest go bounds,I dive many time for the ball manage to save t...,dived many times ball managed save rest bounds,i dived many times for the ball managed to sav...,i dived many times for the ball managed to sav...,i dived many times for the ball managed to sav...
3,0,my whole body feels itchy and like its on fire,negative,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,whole body feels itchy like fire,my whole body feel itchy and like its on fire,whole body feels itchy like fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....",negative,"no, it's not behaving at all. i'm mad. why am ...",no its not behaving at all im mad why am i her...,no its not behaving at all im mad why am i her...,behaving im mad cant see,no its not behave at all I m mad why be I here...,behaving here because cant over there,no its not behaving at all im mad why am i her...,no its not behaving at all i am mad why am i h...,no its not behaving at all im mad why am i her...


In [155]:
sentiment140Df.to_csv("sentiment140PreprocessingExperiment.csv", index=False)